# Коеффициент перехода от кода высокого 255 к коду 250

По данным, полученным с 20:17 по 20:36 08.11.2018г. п. Научный КрАО.  

In [1]:
!date
!pwd
## Чт ноя 15 13:54:55 MSK 2018
## /home/sphere/ExpData/Program/Sketch/Data

Чт ноя 15 19:45:55 MSK 2018
/home/sphere/ExpData/Program/Sketch/Data


In [2]:
import pandas as pd
import numpy as np

In [3]:
"""  Читаем файл с логами """
## ['date', 'time', 'On', 'u', 'i', 't', 'u5', 'dac', 'CH0', 'CH1', 'CH2', 'CH3', 'U']
dtype = {'date': str, 'time': str, 'On': str, 'u': str, 'i': str, 't': str, 'u5': str, 'dac': str,
         'CH0': np.uint8, 'CH1': np.uint8, 'CH2': np.uint8, 'CH3': np.uint8}
data = pd.read_csv("2018_11_08_Voltage.txt", index_col=False, sep='\\s+')

data = data[~data.On.isin(['OFF'])]
data = data[~data.date.isin(['OK','>','Recv','ADC_on:','DAC_write', 'ADC_off:','SEND','Start'])] 
data = data[~data.date.str.contains(r'AT+')]
data = data[~data.date.str.contains(r'SEN')]

## add columns
data["U"]  = data.CH0 * 0.01528 - 36.91 - 2.048
data["I"]  = (4094.0 - data.CH1)/10.372
data["U5"] = (data.CH2 + 20.0)*0.01
data["T"]  = (data.CH3/2.0 - 500) * 0.1

data = data.drop(columns = "adc")
data.head()

,date,time,On,u,i,t,u5,dac,CH0,CH1,CH2,CH3,U,I,U5,T
0,Input:,6:w_210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,08.11.2018,19:28:41,ON,-U=-23.7V,I=4.9uA,T=13.6C,Us=4.9V,DAC:210,863.0,4043.0,474.0,1272.0,-25.77136,4.917084,4.94,13.6
7,08.11.2018,19:28:42,ON,-U=-23.7V,I=4.8uA,T=13.6C,Us=4.9V,DAC:210,863.0,4044.0,474.0,1272.0,-25.77136,4.820671,4.94,13.6
8,08.11.2018,19:28:43,ON,-U=-23.7V,I=5.0uA,T=13.6C,Us=4.9V,DAC:210,863.0,4042.0,474.0,1272.0,-25.77136,5.013498,4.94,13.6
9,08.11.2018,19:28:44,ON,-U=-23.7V,I=4.8uA,T=13.6C,Us=4.9V,DAC:210,863.0,4044.0,474.0,1272.0,-25.77136,4.820671,4.94,13.6


In [4]:
""" Читаем файл расписания с временами и напряжениями """
table = pd.read_csv('Voltage_table.txt', index_col=False, sep='\s+')
print(table.shape)
#table

(7, 3)


In [5]:
""" Читаем файл расписания с временами и напряжениями """
for i in range(1,table.shape[0]):
    part=data[(data.time >= table.begin[i]) & (data.time <= table.end[i])]
    #print(i, part.U.count(), part.U.mean(), sep='\t')
    table.loc[i, 'count'] = part.U.count()                 ## количество измерений (секунд)
    table.loc[i, 'U']     = part.U.mean()                  ## среднее напряжения источника -2.048
    table.loc[i, 'Ustd']  = part.U.std()                   ## станд. отклонение напряжения источника
    table.loc[i, 'I']     = part.I.mean()                  ## ток
    table.loc[i, 'Istd']  = part.I.std()
    table.loc[i, 'T']     = part['T'].mean()               ## температура
    table.loc[i, 'Tstd']  = part['T'].std()
    table.loc[i, 'index0'] = part.index[0]                 ## номер первого кадра в серии
    table.loc[i, 'index1'] = part.index[part.U.count()-1]  ## номер последнего кадра

Таблица характеристик серий измерений

    name      --- комментарий к периоду измерений
    w         --- код DAC высокого напряжения
    begin     --- время начала периода измерений
    end       --- время окончания периода измерений
    count     --- количество измерений (секунд)
    U, Ustd   --- среднее и станд. отклонение напряжения источника  
    I, Istd   --- то же для тока SiPM
    T, Tstd   --- то же для температуры

In [6]:
table

,w,begin,end,count,U,Ustd,I,Istd,T,Tstd,index0,index1
0,off,20:17:05,20:19:04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,210,20:19:18,20:19:48,30.0,-25.780528,0.007614,5.260959,0.159499,8.600000,0.032163,2574.0,2603.0
2,240,20:19:49,20:20:16,27.0,-27.975076,0.006474,24.728257,0.432309,8.598148,0.021845,2610.0,2636.0
3,245,20:20:17,20:21:08,50.0,-28.385462,0.004187,30.622831,0.619001,8.562000,0.025873,2644.0,2693.0
4,250,20:21:09,20:21:59,49.0,-28.809897,0.005402,36.058619,1.032419,8.537755,0.026082,2700.0,2748.0
5,255,20:22:00,20:23:03,62.0,-29.240659,0.003306,41.530859,0.573104,8.511290,0.027786,2756.0,2817.0
6,210,20:23:05,20:35:59,746.0,-25.783670,0.006254,5.396308,0.143368,8.291354,0.111257,2824.0,3569.0


## Результат:

отношение токов при коде высокого 250 и 255 равно 0.868

In [7]:
print(table.I[4] / table.I[5])

0.868236791852
